In [2]:
!nvidia-smi

Sun Jan  9 14:46:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 472.12       Driver Version: 472.12       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| 75%   63C    P2   195W / 200W |   7858MiB /  8192MiB |     93%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

In [4]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
class_names = info.features["label"].names
n_classes = info.features["label"].num_classes
dataset_size = info.splits["train"].num_examples
print(class_names, n_classes, dataset_size)

test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    name="tf_flowers",
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling tf_flowers-train.tfrecord...:   0%|          | 0/3670 [00:00<?, ? examples/s]

Dataset tf_flowers downloaded and prepared to C:\Users\kun\tensorflow_datasets\tf_flowers\3.0.1. Subsequent calls will reuse this data.
['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses'] 5 3670


In [5]:
def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label


batch_size = 32
train_set = train_set_raw.shuffle(1000).repeat().map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

In [6]:
model = keras.Sequential(layers=[
    keras.applications.xception.Xception(include_top=False, weights="imagenet"),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(n_classes, activation=keras.activations.softmax)
])

for layer in model.layers[0].layers:
    layer.trainable = True

lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.01, decay_steps=10000, decay_rate=0.9)

model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.SGD(momentum=0.9, learning_rate=lr_schedule),
              metrics=[keras.metrics.sparse_categorical_accuracy])

model.summary()

history = model.fit(
    train_set,
    steps_per_epoch=int(0.75 * dataset_size / batch_size),
    validation_data=valid_set,
    validation_steps=int(0.15 * dataset_size / batch_size),
    epochs=5
)

83697664/83683744 [==============================] - 36s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, None, None, 2048)  20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 5)                 10245     
                                                                 
Total params: 20,871,725
Trainable params: 20,817,197
Non-trainable params: 54,528
_________________________________________________________________
Epoch 1/5
86/86 [==============================] - 107s 982ms/step - loss: 0.5889 - sparse_categorical_accuracy: 0.7823 - val_loss: 0.2814 - val_sparse_categorical_accurac